# Examining irrigated area in the Nile Valley

## Import modules

In [ ]:
%matplotlib inline
import datacube.utils.rio
import datacube
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd
import scipy
from datacube.helpers import write_geotiff
import sys
from skimage import filters
from datacube.storage.masking import make_mask
from skimage import exposure
from xr_geomedian import xr_geomedian
%load_ext autoreload
%autoreload 2

sys.path.append('../../examples/Scripts')
import deafrica_plotting
import deafrica_datahandling

import os
import dask
from datacube.utils.dask import start_local_dask
from datacube.utils.rio import configure_s3_access

import warnings
warnings.filterwarnings("ignore")

#This will speed up loading data
datacube.utils.rio.set_default_rio_config(aws='auto', cloud_defaults=True)

In [ ]:
dask.config.set({"distributed.dashboard.link": 
        os.environ.get('JUPYTERHUB_SERVICE_PREFIX', '/')+"proxy/{port}/status"})

client = start_local_dask(n_workers=2, threads_per_worker=2, memory_limit='8GB')
display(client)

creds = configure_s3_access(client=client, 
                            region_name='auto')

## Connect to the datacube

In [ ]:
dc = datacube.Datacube(app='test_collections')


## User inputs

In [ ]:
lat =  24.50 #senegal 16.4558  
lon =  33.0 # -15.733
buff = 0.1
time = ('2017-06', '2019-02')
res = (-30, 30)

### View the selected location

In [ ]:
# deafrica_plotting.display_map(x=(lon-buff, lon+buff+0.275), y=(lat+buff, lat-buff))

## Load data and cloud mask

In [ ]:
# Create a query object
queryC2 = {
    'x': (lon-buff, lon+buff),
    'y': (lat+buff, lat-buff),
    'time': (time),
    'resolution': res,
    'group_by': 'solar_day'
}

In [ ]:
# find most common crs to allow
# native loading with correct crs
crs = deafrica_datahandling.mostcommon_crs(dc=dc,
                                          product='usgs_ls8c_level2_2',
                                          query=queryC2)

In [ ]:
#load data
col2 = deafrica_datahandling.load_masked_usgs(dc=dc,
                                            products=['usgs_ls8c_level2_2'],
                                            output_crs=crs,
                                            measurements=['red', 'green', 'blue'], #'surface_temperature'
                                            dask_chunks = {'x':250, 'y':250},
                                            lazy_load=True,
                                            align=(15, 15),
                                            **queryC2)

# col2 = dc.load(product='usgs_ls8c_level2_2',
#                **queryC2,
#                output_crs=crs,
#                measurements=['red', 'green', 'blue', 'quality_l2_aerosol'],
#                align=(15, 15),
#                dask_chunks={'x':250, 'y':250})

# valid_data_C2 = make_mask(col2["quality_l2_aerosol"],
#                           cloud_shadow="not_cloud_shadow",
#                           cloud_or_cirrus="not_cloud_or_cirrus")


# col2 = col2.where(valid_data_C2)

In [ ]:
# dask.visualize(col2)

## Calculate irrigated area using seasonal maximum ndvi


In [ ]:
#calculate ndvi
ndvi = ((col2.nir - col2.red)/(col2.nir + col2.red))

#calculate seasonal max ndvi
ndvi_smax = ndvi.resample(time='QS-JUN').max('time')
ndvi_smax = ndvi_smax.fillna(0)

In [ ]:
ndvi_smax = ndvi_smax.compute()

In [ ]:
# thresholds = [0.2, 0.25, 0.30, 0.35, 0.40]
thresholds = [0.30]

#create padas dataframe for adding results too
data_dict = {'Season': ndvi_smax.time.values}
irr_area = pd.DataFrame(data=data_dict).set_index('Season')

# Loop through thresholds and seasons and calculate area above threshold
for thres in thresholds:
    print("working on threshold "+ str(thres))
    
    areas = []
    for i in range(len(ndvi_smax.time)):
        
        # Convert our mean NDVI xarray into a numpy array
        input_array = ndvi_smax.isel(time=i).values
        
        irr = input_array >= thres
        area = np.count_nonzero(irr) * (30*30) / 10000
        areas.append(area)
        
    name=str(thres)
    irr_area[name] = areas


### Plot irrigated time series

In [ ]:
styles=['bs--', 'ro-', 'y^-', 'go-', 'rs-']
irr_area.plot(figsize=(16,8), style = styles, legend=False)

## Animations

###  line plot animation of irrigation

In [ ]:
# irr_area.columns(['Season', ('Ha')])

In [ ]:
deafrica_plotting.animated_timeseriesline(
    ndvi_smax.fillna(0).rename("Seasonal Maximum NDVI").to_dataset(), #.rolling(time=4, center=True, min_periods=1).mean()
    irr_area,
    "egypt_seasonal_irrigation_cividis.gif",
    width_pixels=1000,
    interval=300,
    bands=["Seasonal Maximum NDVI"],
    onebandplot_cbar=False,
    show_date=False,
    title="Seasonal Irrigated Area (ha)",
    onebandplot_kwargs={"cmap": "cividis", "vmax":"0.55"},
    pandasplot_kwargs={"legend":False, "style": "bs--"},
)

### True colour animation

In [ ]:
%%time

rgb = col2.resample(time='QS-JUN').apply(xr_geomedian)


In [ ]:
deafrica_plotting.rgb(rgb, col='time')

In [ ]:
deafrica_plotting.animated_timeseries(rgb, 'rgb.gif',
                                      width_pixels=1000, interval=300,
                                      bands=['red', 'green', 'blue'],
                                      percentile_stretch=(0.01, 0.99),
                                      annotation_kwargs={'fontsize': 33},
                                      title='Kom Umbu, Egypt',
                                      show_date=True
                                      )


## Experimenting with LST

In [ ]:
#calculate mean seasonal surface temp 
ST_mean = col2.surface_temperature.resample(time='QS-JUN').mean('time')
ST_mean = ST_mean.fillna(0) #need to get rid of nans

#calculate seasonal max ndvi
ndvi = ((col2.nir - col2.red)/(col2.nir + col2.red))
ndvi_smax = ndvi.resample(time='QS-JUN').max('time')
ndvi_smax = ndvi_smax.fillna(0)

In [ ]:
# ndvi_smax = ndvi_smax.compute()
ST_mean = ST_mean.compute()

In [ ]:
areas_LST=[]
for i in range(len(ST_mean.time)):
    #use OTSU to dynamically find thresholds
    val_temp = filters.threshold_otsu(ST_mean.isel(time=i).values)
#     val_ndvi = filters.threshold_otsu(ndvi_smax.isel(time=i).values)
#     print(val_temp)
    
    #use thresholds to classify images
    crop = ndvi_smax.isel(time=i).values >= 0.3
    wet = ST_mean.isel(time=i).values <= val_temp
    irr = crop & wet
    
    #count area irrigated
    area = np.count_nonzero(irr) * (30*30) / 10000
    areas_LST.append(area)
    

In [ ]:
#create padas dataframe for adding results too
data_dict = {'Season': ndvi_smax.time.values, 'Irrigated Area (ha)': areas_LST}
irr_area_LST = pd.DataFrame(data=data_dict).set_index('Season')

In [ ]:
irr_area.plot(figsize=(10,5), style = 'bs-')

In [ ]:
print(np.count_nonzero(crop))
print(np.count_nonzero(wet))
print(np.count_nonzero(irr))

In [ ]:
# Export the array
# Plot to see result
# x = xr.open_rasterio('results_sica/2018-12-01_0.5.tif').squeeze()
# plt.figure(figsize=(20,20))
# plt.imshow(irr, cmap='gist_earth_r', vmin=0.0, vmax=0.6)
# plt.colorbar(shrink=0.5)



# name = str(ndvi_smax.time.values[i])[:10] + "_" + str(thres)
# dea_datahandling.array_to_geotiff('results_sica/201812_rgb.tif',
#                                   ndvi_smax.isel(time=22).values,
#                                   geo_transform=transform,
#                                   projection=projection,
#                                   nodata_val=np.nan)


# transform = col2.geobox.transform.to_gdal()
# projection = col2.geobox.crs.wkt

#         # Calculate the segments
#         segments = quickshift(input_array,
#                               kernel_size=1,
#                               convert2lab=False,
#                               max_dist=10,
#                               ratio=0.5)
        
#         # Calculate the zonal mean NDVI across the segments
#         segments_zonalMean = scipy.ndimage.mean(input=input_array,
#                                                     labels=segments,
#                                                     index=segments)
        
#         irr = segments_zonalMean >= thres

#calculate yearly mas ndvi
# ndvi_max = ndvi.groupby('time.year').max('time').rename('ndvi_max')

In [ ]:
    
#     print(val)
#     hist, bins_center = exposure.histogram(ST_mean.isel(time=i).values)

#     plt.figure(figsize=(10, 5))
#     plt.subplot(121)
#     plt.imshow(ST_mean.isel(time=i).values < val, cmap='gray', interpolation='nearest')
#     plt.subplot(122)
#     plt.plot(bins_center, hist, lw=2)
#     plt.axvline(val, color='k', ls='--')

#     plt.tight_layout()
#     plt.show()